# 4colab

In [1]:
# # Load the Drive helper and mount
# from google.colab import drive

# # This will prompt for authorization.
# drive.mount('/content/drive')

# %cd '/content/drive/MyDrive/Colab Notebooks/C73_A2_1/data'
# !ls

# data_dir = '/content/drive/MyDrive/Colab Notebooks/C73_A2_1/data'
# result_dir = '/content/drive/MyDrive/Colab Notebooks/C73_A2_1/result'

In [2]:
data_dir = 'data'
result_dir = 'result'

# Import

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import copy

import warnings
warnings.filterwarnings("ignore")

# Read data

In [4]:
af = 'vv'
pf = '__1__'

Xtrain = np.load(f'{data_dir}/{af}3.tr.X.{pf}.npy', allow_pickle=True)
y_train = np.load(f'{data_dir}/{af}3.tr.y.npy', allow_pickle=True)

Xtest = np.load(f'{data_dir}/{af}3.t.X.{pf}.npy', allow_pickle=True)
y_test = np.load(f'{data_dir}/{af}3.t.y.npy', allow_pickle=True)

# Xval = np.load(f'{data_dir}/{af}3.v.X.{pf}.npy', allow_pickle=True)
# y_val = np.load(f'{data_dir}/{af}3.v.y.npy', allow_pickle=True)

#? features names
fts_names = [line.strip() for line in open(f'{data_dir}/{af}3.fts_cols.{pf}.txt').readlines()]

In [5]:
Xtrain.shape

(1529165, 63)

In [6]:
# for i in range(0, len(fts_names)):
#     print(i, fts_names[i])

# Test and evaluate

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, PrecisionRecallDisplay, f1_score, roc_auc_score, roc_curve, auc

def evaluate(y_true, y_pred):
    #? relabel to use classification metrics
    y_pred[y_pred != -1] = 0
    y_pred[y_pred == -1] = 1

    cm = confusion_matrix(y_true, y_pred)
    print(cm)
    print(classification_report(y_true, y_pred, digits=4))
    roc_auc = roc_auc_score(y_true, y_pred)
    print(roc_auc)

    # display = PrecisionRecallDisplay.from_predictions(y_true, y_pred)
    # _ = display.ax_.set_title("2-class Precision-Recall curve")
    return roc_auc

In [40]:
import pickle

def save_result(model, Xtr, Xt, model_name, selected_fts_names, roc_auc, expname='', config=None):
    ra = '{:.2f}'.format(round(roc_auc * 100, 2))
    # if ra <= 70: #? not good enough to even bother
    #     return
    
    #? save model
    pickle.dump(model, open(f'{result_dir}/{af}5.{pf}.{model_name}.{expname}.__{ra}__.model.pkl','wb'))

    #? save transformed X as well
    np.save(f'{result_dir}/{af}5.{pf}.{model_name}.{expname}.__{ra}__.data.tr.X.npy', Xtr)
    np.save(f'{result_dir}/{af}5.{pf}.{model_name}.{expname}.__{ra}__.data.t.X.npy', Xt)
    # np.save(f'{result_dir}/{af}5.{pf}.{model_name}.{expname}.__{ra}__.data.v.X.npy', Xv)

    #? save fts
    open(f'{result_dir}/{af}5.{pf}.{model_name}.{expname}.__{ra}__.fts.txt', 'w').write('\n'.join(selected_fts_names))
    
    #? save config
    if config is not None:
        open(f'{result_dir}/{af}5.{pf}.{model_name}.{expname}.__{ra}__.config.txt', 'w').write(config)

## Exp 01: Run on original numeric features (0-8)

`Dur`, `sTos`, `dTos`, `Sport`, `Dport`, `TotPkts`, `TotBytes`, `SrcBytes`

Using only numeric features from original data, the result is not good

```
CPU times: total: 1min 38s
Wall time: 27.7 s

CPU times: total: 9.45 s
Wall time: 9.46 s

[[752381   9199]
 [  2687    275]]
              precision    recall  f1-score   support

           0     0.9964    0.9879    0.9922    761580
           1     0.0290    0.0928    0.0442      2962

    accuracy                         0.9845    764542
   macro avg     0.5127    0.5404    0.5182    764542
weighted avg     0.9927    0.9845    0.9885    764542

0.540381918882559
```

In [21]:
""" select features """
selected_fts_names__01 = fts_names[0:8]
', '.join(selected_fts_names__01)

'Dur, sTos, dTos, Sport, Dport, TotPkts, TotBytes, SrcBytes'

In [22]:
X_train__01 = Xtrain[:,0:8]
X_test__01 = Xtest[:,0:8]
# X_val__01 = Xval[:,0:8]

print(X_train__01.shape)

(1529165, 8)


In [23]:
""" train """
from sklearn.ensemble import IsolationForest

n_estimators = 40
contamination = 0.01
model__01 = IsolationForest(n_estimators=n_estimators, max_samples=X_train__01.shape[0], contamination=contamination, n_jobs=10, verbose=1)

%time model__01.fit(X_train__01)

[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   2 out of  10 | elapsed:    7.4s remaining:   30.1s
[Parallel(n_jobs=10)]: Done  10 out of  10 | elapsed:    8.7s finished


CPU times: total: 1min 38s
Wall time: 27.7 s


IsolationForest(contamination=0.01, max_samples=1529165, n_estimators=40,
                n_jobs=10, verbose=1)

In [24]:
%%time
""" predict """
y_pred__01 = model__01.predict(X_test__01)

CPU times: total: 9.45 s
Wall time: 9.46 s


In [25]:
""" evaluate """
roc_auc__01 = evaluate(y_test, y_pred__01)

[[752381   9199]
 [  2687    275]]
              precision    recall  f1-score   support

           0     0.9964    0.9879    0.9922    761580
           1     0.0290    0.0928    0.0442      2962

    accuracy                         0.9845    764542
   macro avg     0.5127    0.5404    0.5182    764542
weighted avg     0.9927    0.9845    0.9885    764542

0.540381918882559


In [14]:
""" save model """
save_result(model__01, X_train__01, X_test__01, 'iforest', selected_fts_names__01, roc_auc__01, 'exp01_fts-0-8', f'n_estimators={n_estimators}\ncontamination={contamination}')

In [ ]:
""" clean """
del X_train__01, X_test__01#, X_val__01
del model__01, y_pred__01#, yval_pred__01

## Exp 02: Run on numeric features (0-14)

`Dur`, `sTos`, `dTos`, `Sport`, `Dport`, `PktsPerSec`, `BytesPerSec`, `SrcBytesPerSec`, `BytesPerPkt`, `TotPkts`, `TotBytes`, `SrcBytes`, `DstBytes`, `DstBytesPerSec`

Using only numeric features, the result is not good.  
The result does not change much from Exp01

```
CPU times: total: 1min 20s
Wall time: 27.4 s

CPU times: total: 9.98 s
Wall time: 10.1 s

[[753435   8145]
 [  2505    457]]
              precision    recall  f1-score   support

           0     0.9967    0.9893    0.9930    761580
           1     0.0531    0.1543    0.0790      2962

    accuracy                         0.9861    764542
   macro avg     0.5249    0.5718    0.5360    764542
weighted avg     0.9930    0.9861    0.9894    764542

0.5717963861476439
```

In [91]:
""" select features """
selected_fts_names__02 = fts_names[0:14]
', '.join(selected_fts_names__02)

'Dur, sTos, dTos, Sport, Dport, PktsPerSec, BytesPerSec, SrcBytesPerSec, BytesPerPkt, TotPkts, TotBytes, SrcBytes, DstBytes, DstBytesPerSec'

In [92]:
X_train__02 = Xtrain[:,0:14]
X_test__02 = Xtest[:,0:14]
# X_val__02 = Xval[:,0:14]

print(X_train__02.shape)

(1529165, 14)


In [106]:
""" train """
from sklearn.ensemble import IsolationForest

n_estimators = 35
contamination = 0.01
model__02 = IsolationForest(n_estimators=n_estimators, max_samples=X_train__02.shape[0], contamination=contamination, n_jobs=10, verbose=1)

%time model__02.fit(X_train__02)

[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   2 out of  10 | elapsed:    4.8s remaining:   19.7s
[Parallel(n_jobs=10)]: Done  10 out of  10 | elapsed:    7.2s finished


CPU times: total: 1min 20s
Wall time: 27.4 s


IsolationForest(contamination=0.01, max_samples=1529165, n_estimators=35,
                n_jobs=10, verbose=1)

In [107]:
%%time
""" predict """
y_pred__02 = model__02.predict(X_test__02)

CPU times: total: 9.98 s
Wall time: 10.1 s


In [108]:
""" evaluate """
roc_auc__02 = evaluate(y_test, y_pred__02)

[[753435   8145]
 [  2505    457]]
              precision    recall  f1-score   support

           0     0.9967    0.9893    0.9930    761580
           1     0.0531    0.1543    0.0790      2962

    accuracy                         0.9861    764542
   macro avg     0.5249    0.5718    0.5360    764542
weighted avg     0.9930    0.9861    0.9894    764542

0.5717963861476439


In [109]:
""" save model """
save_result(model__02, X_train__02, X_test__02, 'iforest', selected_fts_names__02, roc_auc__02, 'exp02_fts-0-12', f'n_estimators={n_estimators}\ncontamination={contamination}')

In [ ]:
""" clean """
del X_train__02, X_test__02#, X_val__02
del model__02, y_pred__02#, yval_pred__02

## Exp 03: Run on all features

`Dur`, `sTos`, `dTos`, `Sport`, `Dport`, `TotPkts`, `TotBytes`, `SrcBytes`, `PktsPerSec`, `BytesPerSec`, `SrcBytesPerSec`, `BytesPerPkt`, `DstBytes`, `DstBytesPerSec`, `State_CON`, `State_alltcp`, `State_INT`, `State_S_`, `State_URP`, `State_ECO`, `State_RED`, `State_REQ`, `State_ECR`, `State_URH`, `State_TXD`, `State_URFIL`, `State_R_`, `State_URN`, `State_RSP`, `State_URHPRO`, `State_A_`, `State_other`, `Flag_nan`, `Flag_S`, `Flag_A`, `Flag_P`, `Flag_R`, `Flag_F`, `Proto_udp`, `Proto_tcp`, `Proto_icmp`, `Proto_rtp`, `Proto_rtcp`, `Proto_igmp`, `Proto_arp`, `Proto_other`, `Service_80`, `Service_443`, `Service_21`, `Service_22`, `Service_25`, `Service_6667`, `Service_other`

The result increase, yet running time is much higher

```
CPU times: total: 2min 27s
Wall time: 46.4 s

CPU times: total: 16 s
Wall time: 16.2 s

[[750938  10642]
 [  1205   1757]]
              precision    recall  f1-score   support

           0     0.9984    0.9860    0.9922    761580
           1     0.1417    0.5932    0.2288      2962

    accuracy                         0.9845    764542
   macro avg     0.5701    0.7896    0.6105    764542
weighted avg     0.9951    0.9845    0.9892    764542

0.7896033511765821
```

In [19]:
"', '".join(fts_names)

"Dur', 'sTos', 'dTos', 'Sport', 'Dport', 'PktsPerSec', 'BytesPerSec', 'SrcBytesPerSec', 'BytesPerPkt', 'TotPkts', 'TotBytes', 'SrcBytes', 'DstBytes', 'DstBytesPerSec', 'State_CON', 'State_alltcp', 'State_INT', 'State_S_', 'State_URP', 'State_ECO', 'State_RED', 'State_REQ', 'State_ECR', 'State_URH', 'State_TXD', 'State_URFIL', 'State_R_', 'State_URN', 'State_RSP', 'State_URHPRO', 'State_A_', 'State_other', 'Flag_nan', 'Flag_S', 'Flag_A', 'Flag_P', 'Flag_R', 'Flag_F', 'Proto_udp', 'Proto_tcp', 'Proto_icmp', 'Proto_rtp', 'Proto_rtcp', 'Proto_igmp', 'Proto_arp', 'Proto_other', 'Service_80', 'Service_443', 'Service_21', 'Service_22', 'Service_25', 'Service_6667', 'Service_other', 'sTos_0', 'sTos_2', 'sTos_3', 'sTos_1', 'sTos_-1', 'dTos_0', 'dTos_-1', 'dTos_3', 'dTos_2', 'dTos_1"

In [20]:
""" select features """
selected_fts_names__03 = ['Dur', 'sTos', 'dTos', 'Sport', 'Dport', 'TotPkts', 'TotBytes', 'SrcBytes', 'PktsPerSec', 'BytesPerSec', 'SrcBytesPerSec', 'BytesPerPkt', 'DstBytes', 'DstBytesPerSec', 'State_CON', 'State_alltcp', 'State_INT', 'State_S_', 'State_URP', 'State_ECO', 'State_RED', 'State_REQ', 'State_ECR', 'State_URH', 'State_TXD', 'State_URFIL', 'State_R_', 'State_URN', 'State_RSP', 'State_URHPRO', 'State_A_', 'State_other', 'Flag_nan', 'Flag_S', 'Flag_A', 'Flag_P', 'Flag_R', 'Flag_F', 'Proto_udp', 'Proto_tcp', 'Proto_icmp', 'Proto_rtp', 'Proto_rtcp', 'Proto_igmp', 'Proto_arp', 'Proto_other', 'Service_80', 'Service_443', 'Service_21', 'Service_22', 'Service_25', 'Service_6667', 'Service_other']
idx__selected_fts_names__03 = [fts_names.index(s) for s in selected_fts_names__03]
len(idx__selected_fts_names__03)

53

In [10]:
X_train__03 = Xtrain[:,idx__selected_fts_names__03]
X_test__03 = Xtest[:,idx__selected_fts_names__03]
# X_val__03 = Xval[:,idx__selected_fts_names__03]

print(X_train__03.shape)

(1529165, 60)


In [11]:
""" train """
from sklearn.ensemble import IsolationForest

n_estimators = 35
contamination = 0.01
model__03 = IsolationForest(n_estimators=n_estimators, max_samples=X_train__03.shape[0], contamination=contamination, n_jobs=10, verbose=1)

%time model__03.fit(X_train__03)

[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   2 out of  10 | elapsed:   10.9s remaining:   43.9s
[Parallel(n_jobs=10)]: Done  10 out of  10 | elapsed:   14.4s finished


CPU times: total: 2min 27s
Wall time: 46.4 s


IsolationForest(contamination=0.01, max_samples=1529165, n_estimators=35,
                n_jobs=10, verbose=1)

In [12]:
%%time
""" predict """
y_pred__03 = model__03.predict(X_test__03)

CPU times: total: 16 s
Wall time: 16.2 s


In [13]:
""" evaluate """
roc_auc__03 = evaluate(y_test, y_pred__03)

[[750938  10642]
 [  1205   1757]]
              precision    recall  f1-score   support

           0     0.9984    0.9860    0.9922    761580
           1     0.1417    0.5932    0.2288      2962

    accuracy                         0.9845    764542
   macro avg     0.5701    0.7896    0.6105    764542
weighted avg     0.9951    0.9845    0.9892    764542

0.7896033511765821


In [14]:
""" save model """
save_result(model__03, X_train__03, X_test__03, 'iforest', selected_fts_names__03, roc_auc__03, 'exp03_fts-all', f'n_estimators={n_estimators}\ncontamination={contamination}')

In [ ]:
""" clean """
del X_train__03, X_test__03#, X_val__03
del model__03, y_pred__03#, yval_pred__03

## Exp 04: Run on chosen set of features

`sTos`, `Sport`, `Dport`, `TotPkts`, `TotBytes`, `SrcBytes`, `PktsPerSec`, `BytesPerSec`, `SrcBytesPerSec`, `BytesPerPkt`, `State_CON`, `State_alltcp`, `State_INT`, `State_S_`, `State_URP`, `State_ECO`, `State_other`, `Flag_nan`, `Flag_S`, `Flag_A`, `Flag_P`, `Flag_R`, `Flag_F`, `Proto_udp`, `Proto_tcp`, `Proto_icmp`, `Proto_other`, `Service_80`, `Service_443`, `Service_21`, `Service_22`, `Service_25`, `Service_6667`, `Service_other`

Execution time decrease compared to Exp03.  
ROC_AUC and accuracy increases.  
Recall for minority class increases.  

```
CPU times: total: 2min 28s
Wall time: 36.2 s

CPU times: total: 10.5 s
Wall time: 10.6 s

[[752410   9170]
 [  1036   1926]]
              precision    recall  f1-score   support

           0     0.9986    0.9880    0.9933    761580
           1     0.1736    0.6502    0.2740      2962

    accuracy                         0.9867    764542
   macro avg     0.5861    0.8191    0.6336    764542
weighted avg     0.9954    0.9867    0.9905    764542

0.8190977847166908
```

In [9]:
""" select features """
selected_fts_names__04 = ['dTos', 'Sport', 'Dport', 'TotPkts', 'TotBytes', 'SrcBytes', 'PktsPerSec', 'BytesPerSec', 'SrcBytesPerSec', 'BytesPerPkt', 'State_CON', 'State_alltcp', 'State_INT', 'State_S_', 'State_URP', 'State_ECO', 'State_other', 'Flag_nan', 'Flag_S', 'Flag_A', 'Flag_P', 'Flag_R', 'Flag_F', 'Proto_udp', 'Proto_tcp', 'Proto_icmp', 'Proto_other', 'Service_80', 'Service_443', 'Service_21', 'Service_22', 'Service_25', 'Service_6667', 'Service_other']
idx__selected_fts_names__04 = [fts_names.index(s) for s in selected_fts_names__04]
len(idx__selected_fts_names__04)

34

In [10]:
X_train__04 = np.concatenate(tuple([Xtrain[:,[idx]] for idx in idx__selected_fts_names__04]), axis=1)
X_test__04 = np.concatenate(tuple([Xtest[:,[idx]] for idx in idx__selected_fts_names__04]), axis=1)
# X_val__04 = np.concatenate(tuple([Xval[:,[idx]] for idx in idx__selected_fts_names__04]), axis=1)

print(X_train__04.shape)

(1529165, 34)


In [11]:
""" train """
from sklearn.ensemble import IsolationForest

n_estimators = 35
contamination = 0.01
model__04 = IsolationForest(n_estimators=n_estimators, max_samples=X_train__04.shape[0], contamination=contamination, n_jobs=10, verbose=1)

%time model__04.fit(X_train__04)

[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   2 out of  10 | elapsed:   10.9s remaining:   44.1s
[Parallel(n_jobs=10)]: Done  10 out of  10 | elapsed:   14.2s finished


CPU times: total: 2min 28s
Wall time: 36.2 s


IsolationForest(contamination=0.01, max_samples=1529165, n_estimators=35,
                n_jobs=10, verbose=1)

In [12]:
%%time
""" predict """
y_pred__04 = model__04.predict(X_test__04)

CPU times: total: 10.5 s
Wall time: 10.6 s


In [13]:
""" evaluate """
roc_auc__04 = evaluate(y_test, y_pred__04)

[[752410   9170]
 [  1036   1926]]
              precision    recall  f1-score   support

           0     0.9986    0.9880    0.9933    761580
           1     0.1736    0.6502    0.2740      2962

    accuracy                         0.9867    764542
   macro avg     0.5861    0.8191    0.6336    764542
weighted avg     0.9954    0.9867    0.9905    764542

0.8190977847166908


In [15]:
""" predict and evaluate on val set """
yval_pred__04 = model__04.predict(X_val__04)
_ = evaluate(y_val, yval_pred__04)

[[747044   6080]
 [ 10229   1279]]
              precision    recall  f1-score   support

           0     0.9865    0.9919    0.9892    753124
           1     0.1738    0.1111    0.1356     11508

    accuracy                         0.9787    764632
   macro avg     0.5801    0.5515    0.5624    764632
weighted avg     0.9743    0.9787    0.9764    764632

0.5515335183517559


In [14]:
""" save model """
save_result(model__04, X_train__04, X_test__04, 'iforest', selected_fts_names__04, roc_auc__04, 'exp04_play', f'n_estimators={n_estimators}\ncontamination={contamination}')

In [22]:
""" clean """
del X_train__04, X_test__04#, X_val__04
del model__04, y_pred__04, yval_pred__04

## Exp 05: Run on numeric features chosen from MI + all one-hot features

`dTos`, `Sport`, `BytesPerPkt`, `Dur`, `BytesPerSec`, `DstBytesPerSec`, `PktsPerSec`, `SrcBytesPerSec`, `TotBytes`, `DstBytes`

Result is slightly worse than Exp04

```
CPU times: total: 3min 7s
Wall time: 42.6 s

CPU times: total: 11.6 s
Wall time: 11.6 s

[[752050   9530]
 [  1216   1746]]
              precision    recall  f1-score   support

           0     0.9984    0.9875    0.9929    761580
           1     0.1548    0.5895    0.2453      2962

    accuracy                         0.9859    764542
   macro avg     0.5766    0.7885    0.6191    764542
weighted avg     0.9951    0.9859    0.9900    764542

0.7884765588877837
```

In [63]:
""" select features """
selected_fts_names__05 = ['dTos','Sport','BytesPerPkt','Dur','BytesPerSec','DstBytesPerSec','PktsPerSec','SrcBytesPerSec',
    'State_CON', 'State_alltcp', 'State_INT', 'State_S_',
    'State_URP', 'State_ECO', 'State_RED', 'State_REQ', 'State_ECR',
    'State_URH', 'State_TXD', 'State_URFIL', 'State_R_', 'State_URN',
    'State_RSP', 'State_URHPRO', 'State_A_', 'State_other', 'Flag_nan',

    'Flag_S', 'Flag_A', 'Flag_P', 'Flag_R', 'Flag_F', 

    'Proto_udp', 'Proto_tcp', 'Proto_icmp', 'Proto_rtp', 'Proto_rtcp',
    'Proto_igmp', 'Proto_arp', 'Proto_other', 
    
    'Service_80', 'Service_443', 'Service_21', 'Service_22', 'Service_25', 'Service_6667',
    'Service_other'
]
idx__selected_fts_names__05 = [fts_names.index(s) for s in selected_fts_names__05]
len(idx__selected_fts_names__05)

47

In [64]:
X_train__05 = np.concatenate(tuple([Xtrain[:,[idx]] for idx in idx__selected_fts_names__05]), axis=1)
X_test__05 = np.concatenate(tuple([Xtest[:,[idx]] for idx in idx__selected_fts_names__05]), axis=1)
# X_val__05 = np.concatenate(tuple([Xval[:,[idx]] for idx in idx__selected_fts_names__05]), axis=1)

print(X_train__05.shape)

(1529165, 47)


In [69]:
""" train """
from sklearn.ensemble import IsolationForest

n_estimators = 35
contamination = 0.01
model__05 = IsolationForest(n_estimators=n_estimators, max_samples=X_train__05.shape[0], contamination=contamination, n_jobs=10, verbose=1)

%time model__05.fit(X_train__05)

[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   2 out of  10 | elapsed:   14.6s remaining:   58.5s
[Parallel(n_jobs=10)]: Done  10 out of  10 | elapsed:   18.5s finished


CPU times: total: 3min 7s
Wall time: 42.6 s


IsolationForest(contamination=0.01, max_samples=1529165, n_estimators=35,
                n_jobs=10, verbose=1)

In [70]:
%%time
""" predict """
y_pred__05 = model__05.predict(X_test__05)

CPU times: total: 11.6 s
Wall time: 11.6 s


In [71]:
""" evaluate """
roc_auc__05 = evaluate(y_test, y_pred__05)

[[752050   9530]
 [  1216   1746]]
              precision    recall  f1-score   support

           0     0.9984    0.9875    0.9929    761580
           1     0.1548    0.5895    0.2453      2962

    accuracy                         0.9859    764542
   macro avg     0.5766    0.7885    0.6191    764542
weighted avg     0.9951    0.9859    0.9900    764542

0.7884765588877837


In [72]:
""" save model """
save_result(model__05, X_train__05, X_test__05, 'iforest', selected_fts_names__05, roc_auc__05, 'exp05_mi1', f'n_estimators={n_estimators}\ncontamination={contamination}')

In [74]:
""" clean """
del X_train__05, X_test__05#, X_val__05
del model__05, y_pred__05#, yval_pred__05

## Exp 06: Run on features chosen from MI

`State_S_` ,` dTos` ,` Proto_tcp` ,` Proto_udp` ,` Service_80` ,` Service_other` ,` State_CON` ,` Service_25` ,` Sport` ,` Flag_S` ,` Flag_R` ,` BytesPerPkt` ,` Dur` ,` Service_6667` ,` BytesPerSec` ,` State_alltcp` ,` DstBytesPerSec` ,` Flag_A` ,` PktsPerSec` ,` SrcBytesPerSec`

Result is slightly worse than Exp05

```
CPU times: total: 2min 13s
Wall time: 31.9 s

CPU times: total: 9.8 s
Wall time: 9.8 s

[[752537   9043]
 [  1422   1540]]
              precision    recall  f1-score   support

           0     0.9981    0.9881    0.9931    761580
           1     0.1455    0.5199    0.2274      2962

    accuracy                         0.9863    764542
   macro avg     0.5718    0.7540    0.6102    764542
weighted avg     0.9948    0.9863    0.9901    764542

0.7540224874372283
```

In [44]:
""" select features """
selected_fts_names__06 = ['State_S_','dTos','Proto_tcp','Proto_udp','Service_80','Service_other','State_CON','Service_25','Sport','Flag_S','Flag_R','BytesPerPkt','Dur','Service_6667','BytesPerSec','State_alltcp','DstBytesPerSec','Flag_A','PktsPerSec','SrcBytesPerSec']
idx__selected_fts_names__06 = [fts_names.index(s) for s in selected_fts_names__06]
len(idx__selected_fts_names__06)

20

In [45]:
X_train__06 = np.concatenate(tuple([Xtrain[:,[idx]] for idx in idx__selected_fts_names__06]), axis=1)
X_test__06 = np.concatenate(tuple([Xtest[:,[idx]] for idx in idx__selected_fts_names__06]), axis=1)
# X_val__06 = np.concatenate(tuple([Xval[:,[idx]] for idx in idx__selected_fts_names__06]), axis=1)

print(X_train__06.shape)

(1529165, 20)


In [58]:
""" train """
from sklearn.ensemble import IsolationForest

n_estimators = 35
contamination = 0.01
model__06 = IsolationForest(n_estimators=n_estimators, max_samples=X_train__06.shape[0], contamination=contamination, n_jobs=10, verbose=1)

%time model__06.fit(X_train__06)

[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   2 out of  10 | elapsed:   10.2s remaining:   41.0s
[Parallel(n_jobs=10)]: Done  10 out of  10 | elapsed:   13.1s finished


CPU times: total: 2min 13s
Wall time: 31.9 s


IsolationForest(contamination=0.01, max_samples=1529165, n_estimators=35,
                n_jobs=10, verbose=1)

In [59]:
%%time
""" predict """
y_pred__06 = model__06.predict(X_test__06)

CPU times: total: 9.8 s
Wall time: 9.8 s


In [60]:
""" evaluate """
roc_auc__06 = evaluate(y_test, y_pred__06)

[[752537   9043]
 [  1422   1540]]
              precision    recall  f1-score   support

           0     0.9981    0.9881    0.9931    761580
           1     0.1455    0.5199    0.2274      2962

    accuracy                         0.9863    764542
   macro avg     0.5718    0.7540    0.6102    764542
weighted avg     0.9948    0.9863    0.9901    764542

0.7540224874372283


In [61]:
""" save model """
save_result(model__06, X_train__06, X_test__06, 'iforest', selected_fts_names__06, roc_auc__06, 'exp06_mi2', f'n_estimators={n_estimators}\ncontamination={contamination}')

In [62]:
""" clean """
del X_train__06, X_test__06#, X_val__06
del model__06, y_pred__06#, yval_pred__06

## Exp 07: Run on numeric features chosen from MI + ohe features

Chosen features from MI: `dTos`, `Sport`, `BytesPerPkt`, `Dur`, `BytesPerSec`, `DstBytesPerSec`, `PktsPerSec`, `SrcBytesPerSec`, `State_S_`, `Proto_tcp`, `Proto_udp`, `Service_80`, `Service_other`, `State_CON`, `Service_25`, `Flag_S`

```
CPU times: total: 2min 9s
Wall time: 31.1 s

CPU times: total: 9.16 s
Wall time: 9.16 s

[[753170   8410]
 [  1375   1587]]
              precision    recall  f1-score   support

           0     0.9982    0.9890    0.9935    761580
           1     0.1587    0.5358    0.2449      2962

    accuracy                         0.9872    764542
   macro avg     0.5785    0.7624    0.6192    764542
weighted avg     0.9949    0.9872    0.9906    764542

0.7623718993239099
```

In [73]:
""" select features """
selected_fts_names__07 = ['dTos','Sport','BytesPerPkt','Dur','BytesPerSec','DstBytesPerSec','PktsPerSec','SrcBytesPerSec',
    'State_S_','Proto_tcp','Proto_udp','Service_80','Service_other','State_CON','Service_25','Flag_S'
]
idx__selected_fts_names__07 = [fts_names.index(s) for s in selected_fts_names__07]
len(idx__selected_fts_names__07)

16

In [75]:
X_train__07 = np.concatenate(tuple([Xtrain[:,[idx]] for idx in idx__selected_fts_names__07]), axis=1)
X_test__07 = np.concatenate(tuple([Xtest[:,[idx]] for idx in idx__selected_fts_names__07]), axis=1)
# X_val__07 = np.concatenate(tuple([Xval[:,[idx]] for idx in idx__selected_fts_names__07]), axis=1)

print(X_train__07.shape)

(1529165, 16)


In [84]:
""" train """
from sklearn.ensemble import IsolationForest

n_estimators = 35
contamination = 0.01
model__07 = IsolationForest(n_estimators=n_estimators, max_samples=X_train__07.shape[0], contamination=contamination, n_jobs=10, verbose=1)

%time model__07.fit(X_train__07)

[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   2 out of  10 | elapsed:   10.1s remaining:   40.8s
[Parallel(n_jobs=10)]: Done  10 out of  10 | elapsed:   12.4s finished


CPU times: total: 2min 9s
Wall time: 31.1 s


IsolationForest(contamination=0.01, max_samples=1529165, n_estimators=35,
                n_jobs=10, verbose=1)

In [85]:
%%time
""" predict """
y_pred__07 = model__07.predict(X_test__07)

CPU times: total: 9.16 s
Wall time: 9.16 s


In [86]:
""" evaluate """
roc_auc__07 = evaluate(y_test, y_pred__07)

[[753170   8410]
 [  1375   1587]]
              precision    recall  f1-score   support

           0     0.9982    0.9890    0.9935    761580
           1     0.1587    0.5358    0.2449      2962

    accuracy                         0.9872    764542
   macro avg     0.5785    0.7624    0.6192    764542
weighted avg     0.9949    0.9872    0.9906    764542

0.7623718993239099


In [87]:
""" save model """
save_result(model__07, X_train__07, X_test__07, 'iforest', selected_fts_names__07, roc_auc__07, 'exp07_mi3', f'n_estimators={n_estimators}\ncontamination={contamination}')

In [ ]:
""" clean """
del X_train__07, X_test__07#, X_val__07
del model__07, y_pred__07#, yval_pred__07

## Checking score

The best set of features is of Exp04 and Exp02.  
Use this experiment result to further analyze the scores of outliers

Check `vv2-6.after_predict`